In [15]:
import pandas as pd
import os
from PIL import Image
from tqdm.notebook import tqdm

In [16]:
def cleanUp(dir_path1, dir_path2):
    target_set = set(os.listdir(dir_path1))
    for img_name in os.listdir(dir_path2):
        if img_name in target_set:
            continue
        else:
            os.remove(os.path.join(dir_path2, img_name))

In [20]:
SPLIT = 'val'
DATASET_DIR_MAP = f'dataset/sgdm-googlemap-nolabel-1814/{SPLIT}/map'
DATASET_DIR_SAT = f'dataset/sgdm-googlemap-nolabel-1814/{SPLIT}/rs'
LIST_OUTPUT_DIR = 'list/'
IMAGE_OUTPUT_DIR = 'dataset/sgdm-exec'
TARGET_ZOOM_MIN = 14  # 你可以根据需要修改这个值

In [21]:
if not os.path.exists(IMAGE_OUTPUT_DIR):
    os.makedirs(IMAGE_OUTPUT_DIR)

list_filenames_map = os.listdir(DATASET_DIR_MAP)
#list_filenames_sat = os.listdir(DATASET_DIR_SAT)

print("map:",len(list_filenames_map))
#print("sat:",len(list_filenames_sat))

map: 1473


In [22]:
for filename in tqdm(list_filenames_map):
    # 检查文件是否为 PNG 图像
    if filename.endswith('.png'):
        # 读取图像
        img = Image.open(os.path.join(DATASET_DIR_MAP, filename))
        
        # 检查图像是否为 512 大小
        if img.size == (512, 512):
            # 转换图像大小为 256
            resized_img = img.resize((256, 256))
            
            # 保存转换后的图像
            resized_img.save(os.path.join(IMAGE_OUTPUT_DIR, filename))

In [ ]:
df_filenames_map = pd.DataFrame(list_filenames_map, columns=['filename'])
df_filenames_sat = pd.DataFrame(list_filenames_sat, columns=['filename'])

df = df_filenames_map.merge(df_filenames_sat, on='filename', how='outer', indicator=True)

# 找出只在df_filenames_map中存在的行
df_diff_map = df[df['_merge'] == 'left_only']
# 找出只在df_filenames_sat中存在的行
df_diff_sat = df[df['_merge'] == 'right_only']

print(df_diff_map[['filename']].shape)
print(df_diff_sat[['filename']].shape)

df_diff_map = df_diff_map[['filename']]
df_diff_sat = df_diff_sat[['filename']]

In [ ]:
df_diff_sat.to_csv(os.path.join(LIST_OUTPUT_DIR, f'{SPLIT}_file_names_sats_diff.csv'), index=False)

In [ ]:
# 找出两个DataFrame的交集
df_intersection = df_filenames_map.merge(df_filenames_sat, on='filename', how='inner')
df_intersection.shape


In [ ]:
# 将交集保存为CSV文件
df_intersection.to_csv('intersection_dataframe.csv', index=False)

In [ ]:

# 遍历df_diff_map中的文件名，删除对应的文件
for index, row in tqdm(df_diff_map.iterrows(), total=df_diff_map.shape[0], ncols=80, desc='map'):
    filename = row['filename']
    file_path = os.path.join(DATASET_DIR_MAP, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)
    else:
        tqdm.write(f'error:{filename}')

# 遍历df_diff_sat中的文件名，删除对应的文件
for index, row in tqdm(df_diff_sat.iterrows(), total=df_diff_sat.shape[0], ncols=80, desc='sat'):
    filename = row['filename']
    file_path = os.path.join(DATASET_DIR_SAT, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)
    else:
        tqdm.write(f'error:{filename}')


In [ ]:
# 读取 CSV 文件
df_from_csv = pd.read_csv('list/train_file_names_1814_seq.csv')

# 找出两个 DataFrame 的交集
df_merge = df_intersection.merge(df_from_csv, on='filename', how='outer', indicator=True)

# 找出两个 DataFrame 的差集
df_difference = df_merge[df_merge['_merge'] != 'both']
df_difference = df_difference[['filename']]

df_difference.shape

In [ ]:
df_difference.to_csv('list/intersection_dataframe.csv', index=False)